In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import datetime
import os
import sys
import pandas as pd
import time
sys.path.append('../..')

from src.model.two_steps.model import KNN, WeatherGenerator
from src.validation.validation import loss
import pickle
from experiments.real.config import * 
from tqdm import tqdm
from scipy.stats import wilcoxon
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
import plotly.io as pio
alpha = 0.05

In [2]:
results_dir = os.path.join('..', '..', 'data', 'real', 'results_2018')

In [3]:
src_ids = [
    dir_name for dir_name in sorted(os.listdir(results_dir), reverse=True)
    if dir_name.isdigit() and os.path.exists(os.path.join(results_dir, dir_name, 'metrics_test.pickle'))
    ]
src_ids.sort()
len(src_ids)

65

In [4]:
########################################################################
# %% executando
########################################################################
results  = []
for src_id in tqdm(src_ids):
    # observed
    try:
        with open(os.path.join(results_dir, str(src_id), "metrics_test_new.pickle"), 'rb') as file:
            metrics = pd.DataFrame(pickle.load(file))
        with open(os.path.join(results_dir, str(src_id), "weather_pert_hpt_best.pickle"), 'rb') as file:
            weather_pert_hpt_best = pickle.load(file)
        #print(src_id, len(metrics))
        #print(weather_pert_hpt_best['right-tail'])
        row = {
            'src_id': src_id
        }
        for loss in ['right-tail', 'pinball', 'density']:
            two_steps = metrics.loc[metrics['model'] == 'two_steps', loss].tolist()
            weather_pert = metrics.loc[metrics['model'] == 'weather_pert', loss].tolist()
            statistic, p_value = wilcoxon(two_steps, weather_pert)
            sig = p_value < alpha
            row = {**row, 
                f'{loss}_sig': sig,
                f'two_steps_{loss}_mean': np.mean(two_steps),
                f'two_steps_{loss}_std': np.std(two_steps),
                f'two_steps_{loss}_mean_error': np.std(two_steps) / np.sqrt(len(two_steps)),
                f'weather_pert_{loss}_mean': np.mean(weather_pert),
                f'weather_pert_{loss}_std': np.std(weather_pert),
                f'weather_pert_{loss}_mean_error': np.std(weather_pert) / np.sqrt(len(weather_pert)),}
        results.append(row)
    except Exception as e:
        print(e)
        continue
    

results = pd.DataFrame(results)

100%|██████████| 65/65 [00:00<00:00, 122.03it/s]


In [5]:
for loss in ['right-tail', 'pinball', 'density']:
    results[f'best_model_{loss}'] = [('two_steps' if row[f'two_steps_{loss}_mean'] < row[f'weather_pert_{loss}_mean'] else 'weather_pert')  for row in results.to_dict('records')]
    results[f'best_sig_model_{loss}'] = [(row[f'best_model_{loss}'] if row[f'{loss}_sig']  else 'not sig')  for row in results.to_dict('records')]
results.head()

,src_id,right-tail_sig,two_steps_right-tail_mean,two_steps_right-tail_std,two_steps_right-tail_mean_error,weather_pert_right-tail_mean,weather_pert_right-tail_std,weather_pert_right-tail_mean_error,pinball_sig,two_steps_pinball_mean,...,two_steps_density_mean_error,weather_pert_density_mean,weather_pert_density_std,weather_pert_density_mean_error,best_model_right-tail,best_sig_model_right-tail,best_model_pinball,best_sig_model_pinball,best_model_density,best_sig_model_density
0,1033,False,1.612476,0.981766,0.111163,1.480321,0.905745,0.102555,True,5.822259,...,0.001439,-0.028219,0.011750,0.001330,weather_pert,not sig,weather_pert,weather_pert,weather_pert,not sig
1,105,False,1.625576,1.098313,0.122795,1.475766,0.818271,0.091485,True,6.090128,...,0.001127,-0.029395,0.008798,0.000984,weather_pert,not sig,weather_pert,weather_pert,weather_pert,weather_pert
2,1055,True,2.364595,1.817675,0.177387,1.903278,1.142214,0.111469,True,7.671965,...,0.001217,-0.023657,0.011089,0.001082,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
3,113,True,1.589608,0.936909,0.091433,1.313698,0.910599,0.088865,True,5.060084,...,0.001388,-0.038801,0.020881,0.002038,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
4,1137,True,1.995837,1.994950,0.194687,1.740887,1.807490,0.176393,True,6.125418,...,0.002110,-0.035626,0.021501,0.002098,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert


In [6]:
results['best_sig_model'] = [(
    row['best_sig_model_right-tail'] if row['best_sig_model_right-tail'] != 'not sig' else
    row['best_sig_model_pinball'] if row['best_sig_model_pinball'] != 'not sig' else
    row['best_sig_model_density']
    )  for row in results.to_dict('records')]
results[f'best_sig_model'].value_counts()

best_sig_model
weather_pert    34
two_steps       20
not sig         11
Name: count, dtype: int64

In [7]:
results.head()

,src_id,right-tail_sig,two_steps_right-tail_mean,two_steps_right-tail_std,two_steps_right-tail_mean_error,weather_pert_right-tail_mean,weather_pert_right-tail_std,weather_pert_right-tail_mean_error,pinball_sig,two_steps_pinball_mean,...,weather_pert_density_mean,weather_pert_density_std,weather_pert_density_mean_error,best_model_right-tail,best_sig_model_right-tail,best_model_pinball,best_sig_model_pinball,best_model_density,best_sig_model_density,best_sig_model
0,1033,False,1.612476,0.981766,0.111163,1.480321,0.905745,0.102555,True,5.822259,...,-0.028219,0.011750,0.001330,weather_pert,not sig,weather_pert,weather_pert,weather_pert,not sig,weather_pert
1,105,False,1.625576,1.098313,0.122795,1.475766,0.818271,0.091485,True,6.090128,...,-0.029395,0.008798,0.000984,weather_pert,not sig,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
2,1055,True,2.364595,1.817675,0.177387,1.903278,1.142214,0.111469,True,7.671965,...,-0.023657,0.011089,0.001082,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
3,113,True,1.589608,0.936909,0.091433,1.313698,0.910599,0.088865,True,5.060084,...,-0.038801,0.020881,0.002038,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
4,1137,True,1.995837,1.994950,0.194687,1.740887,1.807490,0.176393,True,6.125418,...,-0.035626,0.021501,0.002098,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert


In [13]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=2, shared_xaxes=False, shared_yaxes=False,
    subplot_titles=['', '(a) Pinball cauda direita', '(b) Pinball', '(c) Densidade'],
    horizontal_spacing = 0.15,
    vertical_spacing  = 0.15)

models = {
    'Diferença não significativa': {
        'cor_ponto': 'rgba(148, 148, 148, 0.8)',
        'cor_erro': 'rgba(148, 148, 148, 0.6)',
        'value': 'not sig'
    },
    'Modelo proposto foi melhor': {
        'cor_ponto': 'rgba(254,127,12, 1)',
        'cor_erro': 'rgba(254,127,12, 0.6)',
        'value': 'weather_pert'
    },
    'Gerador climático foi melhor': {
        'cor_ponto': 'rgba(32,120,180, 1)',
        'cor_erro': 'rgba(32,120,180, 0.6)',
        'value': 'two_steps'
    },
}

for (row, col), loss in zip([(1, 2), (2, 1), (2,2)],['right-tail', 'pinball', 'density']):
    print((row, col), loss)
    range = [
        min([(results[f'two_steps_{loss}_mean'] + results[f'two_steps_{loss}_mean_error']).min(), (results[f'weather_pert_{loss}_mean'] + results[f'weather_pert_{loss}_mean_error']).min(), 0]),
        max([(results[f'two_steps_{loss}_mean'] + results[f'two_steps_{loss}_mean_error']).max(), (results[f'weather_pert_{loss}_mean'] + results[f'weather_pert_{loss}_mean_error']).max()])
    ]

    for label, config in models.items():
        _results = results.loc[lambda df: df[f'best_sig_model_{loss}'] == config['value']]
        fig_ = go.Scatter(
            x=_results[f'two_steps_{loss}_mean'],
            y=_results[f'weather_pert_{loss}_mean'],
            mode='markers',
            name=label,
            error_y=dict(
                type='data', # tipo de barra de erro 
                array=_results[f'weather_pert_{loss}_mean_error'], # valor do erro em y para cada ponto (acima)
                arrayminus=_results[f'weather_pert_{loss}_mean_error'], # valor do erro em y para cada ponto (abaixo)
                visible=True, color=config['cor_erro']),
            error_x=dict(
                type='data', # tipo de barra de erro 
                array=_results[f'two_steps_{loss}_mean_error'], # valor do erro em x para cada ponto (direita)
                arrayminus=_results[f'two_steps_{loss}_mean_error'], # valor do erro em x para cada ponto (esquerda)
                visible=True, color=config['cor_erro']),
            marker=dict(
                color=config['cor_ponto'], 
                size=5, line=dict(width=1, color='DarkSlateGrey')
            ),
        )
        if row != 1:
            fig_.update(showlegend=False)

        fig.add_trace(fig_, row=row, col=col)
        

        # Adicione seus traços aqui
    fig_ = go.Scatter(
        x=range,
        y=range,
        mode="lines",
        name="Linha identidade",
        line=dict(color='rgba(0,0,0,0.6)', dash='dash')
    )
    if row != 1:
        fig_.update(showlegend=False)
    fig.add_trace( fig_, row=row, col=col)
    fig.update_xaxes(range=range, row=row, col=col, title="Gerador climático")
    fig.update_yaxes(range=range, row=row, col=col, title="Modelo proposto")

fig.update_layout(legend=dict(orientation='v', yanchor='bottom', y=0.8, x = 0))

fig.update_layout(
    template = "simple_white",
    width=800, height=800,
    margin=dict(t=50))


fig.write_image('exp_comparacao_modelo.pdf')#plot_url = py.plot(fig, filename='latex', include_mathjax='cdn')

(1, 2) right-tail
(2, 1) pinball
(2, 2) density


In [9]:
results.groupby(['best_sig_model_right-tail', 'best_sig_model_pinball', 'best_sig_model_density', 'best_sig_model']).agg(count = ('src_id', 'count'))

count
best_sig_model_right-tail best_sig_model_pinball best_sig_model_density best_sig_model       
not sig                   not sig                not sig                not sig            11
                                                 two_steps              two_steps           1
                                                 weather_pert           weather_pert        3
                          two_steps              not sig                two_steps           2
                                                 weather_pert           two_steps           1
                          weather_pert           not sig                weather_pert        8
                                                 weather_pert           weather_pert        2
two_steps                 not sig                not sig                two_steps           5
                          two_steps              not sig                two_steps           5
                                                 two_steps              two_steps           6
weather_pert              not sig                two_steps              weather_pert        1
                          weather_pert           not sig                weather_pert        1
                                                 weather_pert           weather_pert       19

In [12]:
results

,src_id,right-tail_sig,two_steps_right-tail_mean,two_steps_right-tail_std,two_steps_right-tail_mean_error,weather_pert_right-tail_mean,weather_pert_right-tail_std,weather_pert_right-tail_mean_error,pinball_sig,two_steps_pinball_mean,...,weather_pert_density_mean,weather_pert_density_std,weather_pert_density_mean_error,best_model_right-tail,best_sig_model_right-tail,best_model_pinball,best_sig_model_pinball,best_model_density,best_sig_model_density,best_sig_model
0,1033,False,1.612476,0.981766,0.111163,1.480321,0.905745,0.102555,True,5.822259,...,-0.028219,0.011750,0.001330,weather_pert,not sig,weather_pert,weather_pert,weather_pert,not sig,weather_pert
1,105,False,1.625576,1.098313,0.122795,1.475766,0.818271,0.091485,True,6.090128,...,-0.029395,0.008798,0.000984,weather_pert,not sig,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
2,1055,True,2.364595,1.817675,0.177387,1.903278,1.142214,0.111469,True,7.671965,...,-0.023657,0.011089,0.001082,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
3,113,True,1.589608,0.936909,0.091433,1.313698,0.910599,0.088865,True,5.060084,...,-0.038801,0.020881,0.002038,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
4,1137,True,1.995837,1.994950,0.194687,1.740887,1.807490,0.176393,True,6.125418,...,-0.035626,0.021501,0.002098,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,888,True,1.642928,1.503227,0.146700,2.451463,2.146676,0.209494,True,5.110451,...,-0.033547,0.036817,0.003593,two_steps,two_steps,two_steps,two_steps,two_steps,two_steps,two_steps
61,889,True,1.261704,1.415050,0.138095,1.895055,2.126478,0.207523,True,4.174145,...,-0.039878,0.032583,0.003180,two_steps,two_steps,two_steps,two_steps,two_steps,not sig,two_steps
62,9,True,4.829371,2.644249,0.258052,5.019689,2.715453,0.265001,True,16.722446,...,-0.011979,0.011943,0.001165,two_steps,two_steps,two_steps,two_steps,two_steps,not sig,two_steps
63,908,True,1.592974,0.936059,0.091350,1.608262,0.926271,0.090395,False,5.062485,...,-0.028205,0.011095,0.001083,two_steps,two_steps,two_steps,not sig,two_steps,not sig,two_steps


In [13]:
results
for row in results.to_dict('records'):
    print(
        f"{row['src_id']} & "

        f"{row['weather_pert_right-tail_mean']:.2f} ± {row['weather_pert_right-tail_mean_error']:.2f} {'*' if row['best_sig_model_right-tail'] == 'weather_pert' else ''} & "
        f"{row['two_steps_right-tail_mean']:.2f} ± {row['two_steps_right-tail_mean_error']:.2f} {'*' if row['best_sig_model_right-tail'] == 'two_steps' else ''} & "

        f"{row['weather_pert_pinball_mean']:.2f} ± {row['weather_pert_pinball_mean_error']:.2f} {'*' if row['best_sig_model_pinball'] == 'weather_pert' else ''} & "
        f"{row['two_steps_pinball_mean']:.2f} ± {row['two_steps_pinball_mean_error']:.2f} {'*' if row['best_sig_model_pinball'] == 'two_steps' else ''} & "

        f"{row['weather_pert_density_mean']:.2f} ± {row['weather_pert_density_mean_error']:.2f} {'*' if row['best_sig_model_density'] == 'weather_pert' else ''} & "
        f"{row['two_steps_density_mean']:.2f} ± {row['two_steps_density_mean_error']:.2f} {'*' if row['best_sig_model_density'] == 'two_steps' else ''} & "


        f"{'Modelo proposto' if row['best_sig_model'] == 'weather_pert' else 'Gerador climático' if row['best_sig_model'] == 'two_steps' else ''} \\\\ "
    )

1033 & 1.48 ± 0.10  & 1.61 ± 0.11  & 5.37 ± 0.36 * & 5.82 ± 0.38  & -0.03 ± 0.00  & -0.03 ± 0.00  & Modelo proposto \\ 
105 & 1.48 ± 0.09  & 1.63 ± 0.12  & 4.69 ± 0.27 * & 6.09 ± 0.44  & -0.03 ± 0.00 * & -0.03 ± 0.00  & Modelo proposto \\ 
1055 & 1.90 ± 0.11 * & 2.36 ± 0.18  & 6.57 ± 0.37 * & 7.67 ± 0.49  & -0.02 ± 0.00 * & -0.02 ± 0.00  & Modelo proposto \\ 
113 & 1.31 ± 0.09 * & 1.59 ± 0.09  & 4.56 ± 0.34 * & 5.06 ± 0.34  & -0.04 ± 0.00 * & -0.03 ± 0.00  & Modelo proposto \\ 
1137 & 1.74 ± 0.18 * & 2.00 ± 0.19  & 5.48 ± 0.45 * & 6.13 ± 0.48  & -0.04 ± 0.00 * & -0.03 ± 0.00  & Modelo proposto \\ 
1145 & 1.43 ± 0.11  & 1.44 ± 0.11  & 4.96 ± 0.32  & 5.00 ± 0.35  & -0.03 ± 0.00  & -0.03 ± 0.00  &  \\ 
1161 & 1.89 ± 0.12  & 1.98 ± 0.13  & 6.91 ± 0.37 * & 7.25 ± 0.42  & -0.02 ± 0.00  & -0.02 ± 0.00  & Modelo proposto \\ 
1171 & 3.13 ± 0.24  & 3.29 ± 0.22  & 10.96 ± 0.79 * & 11.46 ± 0.69  & -0.02 ± 0.00  & -0.02 ± 0.00  & Modelo proposto \\ 
1190 & 1.88 ± 0.20 * & 3.07 ± 0.31  & 6.47 ± 0.53

In [23]:
results

,src_id,right-tail_sig,two_steps_right-tail_mean,two_steps_right-tail_std,two_steps_right-tail_mean_error,weather_pert_right-tail_mean,weather_pert_right-tail_std,weather_pert_right-tail_mean_error,pinball_sig,two_steps_pinball_mean,...,weather_pert_density_mean,weather_pert_density_std,weather_pert_density_mean_error,best_model_right-tail,best_sig_model_right-tail,best_model_pinball,best_sig_model_pinball,best_model_density,best_sig_model_density,best_sig_model
0,1033,False,1.612476,0.981766,0.111163,1.480321,0.905745,0.102555,True,5.822259,...,-0.028219,0.011750,0.001330,weather_pert,not sig,weather_pert,weather_pert,weather_pert,not sig,weather_pert
1,105,False,1.625576,1.098313,0.122795,1.475766,0.818271,0.091485,True,6.090128,...,-0.029395,0.008798,0.000984,weather_pert,not sig,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
2,1055,True,2.364595,1.817675,0.177387,1.903278,1.142214,0.111469,True,7.671965,...,-0.023657,0.011089,0.001082,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
3,113,True,1.589608,0.936909,0.091433,1.313698,0.910599,0.088865,True,5.060084,...,-0.038801,0.020881,0.002038,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
4,1137,True,1.995837,1.994950,0.194687,1.740887,1.807490,0.176393,True,6.125418,...,-0.035626,0.021501,0.002098,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert,weather_pert
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,888,True,1.642928,1.503227,0.146700,2.451463,2.146676,0.209494,True,5.110451,...,-0.033547,0.036817,0.003593,two_steps,two_steps,two_steps,two_steps,two_steps,two_steps,two_steps
61,889,True,1.261704,1.415050,0.138095,1.895055,2.126478,0.207523,True,4.174145,...,-0.039878,0.032583,0.003180,two_steps,two_steps,two_steps,two_steps,two_steps,not sig,two_steps
62,9,True,4.829371,2.644249,0.258052,5.019689,2.715453,0.265001,True,16.722446,...,-0.011979,0.011943,0.001165,two_steps,two_steps,two_steps,two_steps,two_steps,not sig,two_steps
63,908,True,1.592974,0.936059,0.091350,1.608262,0.926271,0.090395,False,5.062485,...,-0.028205,0.011095,0.001083,two_steps,two_steps,two_steps,not sig,two_steps,not sig,two_steps
